## XGBoost

Выделяют три группы параметров:
- Общие параметры, отвечающие за базовый алгоритм для бустинга и распараллеливание.
- Параметры выбранного базового алгоритма.
- Параметры обучения, отвечающие за функцию потерь и метрику качества на валидации.

**1. Общие параметры:**
- booster [default=gbtree] - тип базового алгоритма для бустинга: дерево решений gbtree или линейная модель gblinear. 
- silent [default=0] - выдавать (silent=0) или нет (silent=1) сообщения по ходу работы алгоритма.
- nthread [default to maximum number of threads available if not set] - число потоков доступных для параллельной работы xgboost.

**2. Параметры базового алгоритма:**

**2.1. Дерево решений:**
- eta [default=0.3] - темп обучения, перед добавлением дерева в композицию оно умножается на eta. Используется для предотвращения переобучения за счёт "сокращения" весов базовых алгоритмов, делая модель более консервативной. Чем меньше eta, тем больше нужно итераций num_boost_round для обучения модели с хорошим качеством. Диапазон: [0, 1]
- gamma [default=0] - минимальное снижение значения функции потерь, необходимое для дальнейшего разбиения вершины дерева. Большие значения gamma > 0 приводят к более консервативным моделям. Диапазон: [0, $\infty$).
- max_depth [default=6] - максимальная глубина дерева. Диапазон: [1, $\infty$). 
- min_child_weight [default=1] - минимальное необходимое (взвешенное) число примеров в каждой вершине. Чем больше, тем более консервативна итоговая модель. Диапазон: [0, $\infty$).
- max_delta_step [default=0] - обычно равен нулю. Положительные значения используются при несбалансированных классах для ускорения сходимости. Диапазон [0, $\infty$).
- subsample [default=1] - доля выборки, используемая для обучения каждого дерева. Если subsample < 1, то выбирается случайная подвыборка, что помогает в борьбе с переобучением. Диапазон: (0, 1]
- colsample_bytree [default=1] - доля признаков, используемая для обучения каждого дерева. Диапазон: (0, 1]
- lambda [default=1] - коэффициент перед $L_2$-регуляризатором в функции потерь.
- alpha [default=0] - коэффициент перед $L_1$-регуляризатором в функции потерь.

**2.2. Линейная модель:**
- lambda [default=0] - коэффициент перед $L_2$-регуляризатором вектора весов в функции потерь.
- alpha [default=0] - коэффициент перед $L_1$-регуляризатором вектора весов в функции потерь.
- lambda_bias [default=0] - коэффициент перед $L_2$-регуляризатором смещения (свободного члена) в функции потерь.

**3. Параметры задачи обучения:**
- objective [default=reg:linear] - используемая при обучении функция потерь:
    - "reg:linear" – линейная регрессия.
    - "reg:logistic" – логистическая регрессия.
    - "binary:logistic" – логистическая регрессия для бинарной классификации, на выходе - вероятность.
    - "binary:logitraw" – то же самое, но на выходе - значение до его преобразования логистической функцией.
    - "count:poisson" – регрессия Пуассона (используется для оценки числа каких-то событий, счётный признак), на выходе - матожидания распределения Пуассона. В этом случае max_delta_step автоматически устанавливается равным 0.7.
    - "multi:softmax" – обобщение логистической регрессии на многоклассовый случай. При этом нужно задать параметр num_class.
    - "multi:softprob" – то же самое, но на выходе - вектор размера ndata * nclass, который можно преобразовать в матрицу, содержащую вероятности отнесения данного объекта к данному классу.
    - "rank:pairwise" – используется для задач ранжирования.
- base_score [default=0.5] - инициализация значения модели для всех примеров, глобальное смещение.
- eval_metric [default according to objective] - метрика качества на валидационной выборке (по умолчанию соответствует функции потерь: rmse - для регрессии, error - для классификации, mean average precision - для ранжирования). Выбрать можно одну из следующих метрик:
    - "rmse": root mean square error.
    - "logloss": минус логарифм правдоподобия.
    - "error": доля ошибок для бинарной классификации.
    - "merror": то же самое для многоклассовой классификации.
    - "mlogloss": logloss для многоклассовой классификации.
    - "auc": AUC.
    - "ndcg": Normalized Discounted Cumulative Gain.
    - "map": Mean average precision.
    - "ndcg@n",”map@n”: здесь n - целое число, первые n позиций в списке не учитываются.
    - "ndcg-",”map-”,”ndcg@n-”,”map@n-”: списку из всех положительных примеров будет присвоено значение 0 (вместо 1).
- seed [default=0] - для воспроизводимости "случайности".

**Параметры в xgboost.train**:
- params (dict) – параметры, описанные выше.
- dtrain (DMatrix) – обучающая выборка.
- num_boost_round (int) – число итераций бустинга.
- evals (list) – список для оценки качества во время обучения.
- obj (function) – собственная функция потерь.
- feval (function) – собственная функция для оценки качества.
- maximize (bool) – нужно ли максимизировать feval.
- early_stopping_rounds (int) – активирует early stopping. Ошибка на валидации должна уменьшаться каждые early_stopping_rounds итераций для продолжения обучения. Список evals должен быть не пуст. Возвращается модель с последней итерации. Если произошел ранний останов, то модель будет содержать поля: bst.best_score и bst.best_iteration.
- evals_result (dict) – результаты оценки качества.
- verbose_eval (bool) – вывод значения метрики качества на каждой итерации бустинга.
- learning_rates (list or function) – коэффициент скорости обучения для каждой итерации - list l: eta = l[boosting round] - function f: eta = f(boosting round, num_boost_round).
- xgb_model (file name of stored xgb model or ‘Booster’ instance) – возможность продолжить обучения имеющейся модели XGB.


## sklearn.ensemble.GradientBoostingClassifier
- loss [default="deviance"] - оптимизируемая функция потерь.  Одна из {"deviance", "exponential"}. Первая соответствует логистической регрессии и возвращает вероятности, вторая - AdaBoost.
- learning_rate [default=0.1] - темп обучения, аналогично eta для XGBoost.
- n_estimators [default=100] - число итераций градиентного бустинга.
- max_depth [default=3] - аналогично max_depth для XGBoost.
- min_samples_split [default=2] - минимальное число примеров, необходимое для разветвления в данной вершине,  аналогично min_child_weight для XGBoost.
- min_samples_leaf [default=1] - минимальное число примеров в листе.
- min_weight_fraction_leaf [default=0.0] - минимальное взвешенное число примеров в листе.
- subsample [default=1.0] - аналогично subsample для XGBoost.
- max_features (int, float, string or None) [default=None] - число (или доля) признаков, используемых при разбиении вершины.
    - "auto", тогда max_features=sqrt(n_features).
    - "sqrt", тогда max_features=sqrt(n_features).
    - "log2", тогда max_features=log2(n_features).
    - None, тогда max_features=n_features.
- max_leaf_nodes [default=None]
- init (BaseEstimator or None) [default=None] - алгоритм для начальных предсказаний.
- verbose [default=0] - аналогично silent для XGBoost.
- warm_start [default=False] - если True, используется ансамбль с предыдущего вызова fit, новые алгоритмы добавляются к нему, иначе строится новый алгоритм.

Что можно улучшить?
1. Снижение разброса

- Для уменьшения сложности модели можно:

    - использовать меньше признаков (например, отбор)
    - использовать больше объектов (например, искусственно созданных)
    - увеличить регуляризацию

- В случае XGBoost можно:

    - уменьшать максимальную глубину деревьев(max_depth)
    - увеличивать значение параметра min_child_weight
    - увеличивать значение параметра gamma
    - добавлять больше "случайности" за счет параметров subsample и colsample_bytree
    - увеличивать значение параметров регуляризации lambda и alpha

2. Снижение смещения

- Если модель слишком простая, можно:

    - добавлять больше признаков (например, изобретать их),
    - усложнять модель
    - уменьшать регуляризацию

- В случае XGBoost можно:

    - увеличивать максимальную глубину деревьев(max_depth)
    - уменьшать значение параметра min_child_weight
    - уменьшать значение параметра gamma
    - уменьшать значение параметров регуляризации lambda и alpha



## Общие советы

Есть несколько общих советов по работе с несбалансированными выборками:

   - собрать больше данных
   - использовать метрики, нечувствительные к дисбалансу классов (F1, ROC AUC)
   - oversampling/undersampling - брать больше объектов мало представленного класса, и мало - частого класса
   - создать искусственные объекты, похожие на объекты редкого класса (например, алгоритмом SMOTE)

С XGBoost можно:

   - следить за тем, чтобы параметр min_child_weight был мал, хотя по умолчанию он и так равен 1.
   - задать ббольше веса некоторым объектам при инициализации DMatrix
   - контролировать отшошение числа представителей разных классов с помощью параметра set_pos_weight



In [53]:
import numpy as np
import pandas as pd
pd.set_option('display.max.columns', 100)
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv('../data/bank.csv')

In [4]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,0
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,0
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,0
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,0
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        4521 non-null   int64 
 1   job        4521 non-null   object
 2   marital    4521 non-null   object
 3   education  4521 non-null   object
 4   default    4521 non-null   object
 5   balance    4521 non-null   int64 
 6   housing    4521 non-null   object
 7   loan       4521 non-null   object
 8   contact    4521 non-null   object
 9   day        4521 non-null   int64 
 10  month      4521 non-null   object
 11  duration   4521 non-null   int64 
 12  campaign   4521 non-null   int64 
 13  pdays      4521 non-null   int64 
 14  previous   4521 non-null   int64 
 15  poutcome   4521 non-null   object
 16  y          4521 non-null   int64 
dtypes: int64(8), object(9)
memory usage: 600.6+ KB



## Без категориальных признаков¶

Попытаемся сначала просто проигнорировать категориальные признаки. Обучим случайный лес и посмотрим на ROC AUC на кросс-валидации и на отоженной выборке. Это будет наш бейзлайн.


In [6]:
df_no_cat, y = df.loc[:, df.dtypes != 'object'].drop('y', axis=1), df['y']

In [7]:
df_no_cat_part, df_no_cat_valid, y_train_part, y_valid = train_test_split(df_no_cat, y,
                                                                            test_size=0.3, 
                                                                            stratify=y,
                                                                            random_state=17)

In [8]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

In [9]:
forest = RandomForestClassifier(random_state=17)

In [12]:
np.mean(cross_val_score(forest, df_no_cat_part, y_train_part, cv=skf, scoring='roc_auc'))

0.8495016786335677

In [13]:
forest.fit(df_no_cat_part, y_train_part)
roc_auc_score(y_valid, forest.predict_proba(df_no_cat_valid)[:, 1])

0.8631508998911164


## LabelEncoder для категориальных признаков

Сделаем то же самое, но попробуем закодировать категориальные признаки по-простому: с помощью LabelEncoder.

In [14]:
from sklearn.preprocessing import LabelEncoder

In [15]:
label_encoder = LabelEncoder()
df_cat_label_enc = df.copy().drop('y', axis=1)
for col in df.columns[df.dtypes == 'object']:
    df_cat_label_enc[col] = label_encoder.fit_transform(df_cat_label_enc[col])

In [17]:
df_cat_label_enc_part, df_cat_label_enc_valid = train_test_split(df_cat_label_enc, test_size=.3, 
                                                    stratify=y, random_state=17)

In [18]:
np.mean(cross_val_score(forest, df_cat_label_enc_part, y_train_part, cv=skf, scoring='roc_auc'))

0.8922975749958866

In [19]:
forest.fit(df_cat_label_enc_part, y_train_part)
roc_auc_score(y_valid, forest.predict_proba(df_cat_label_enc_valid)[:, 1])

0.908855334230022


## Бинаризация категориальных признаков (dummies, OHE)

Теперь сделаем то, что обычно по умолчанию и делают – бинаризацию категориальных признаков. Dummy-признаки, One-Hot Encoding... с небольшими различиями это об одном же - для каждого значения каждого категориального признака завести свой бинарный признак.


In [20]:
df_cat_dummies = pd.get_dummies(df, columns=df.columns[df.dtypes == 'object']).drop('y', axis=1)

In [21]:
df_cat_dummies_part, df_cat_dummies_valid = train_test_split(df_cat_dummies, test_size=.3, 
                                                    stratify=y, random_state=17)

In [22]:
np.mean(cross_val_score(forest, df_cat_dummies_part, y_train_part, cv=skf, scoring='roc_auc'))

0.8988421716862304

In [23]:
forest.fit(df_cat_dummies_part, y_train_part)
roc_auc_score(y_valid, forest.predict_proba(df_cat_dummies_valid)[:, 1])

0.9172511155233887


## Попарные взаимодействия признаков¶

Пока лес все еще лучше регрессии (хотя мы не тюнили гиперпараметры, но и не будем). Мы хотим идти дальше. Мощной техникой для работы с категориальными признаками будет учет попарных взаимодействий признаков (feature interactions). Построим попарные взаимодействия всех признаков.

In [62]:
df_interact = df.copy()

In [63]:
cat_features = df.columns[df.dtypes == 'object']
for i, col1 in enumerate(cat_features):
    for j, col2 in enumerate(cat_features[i + 1:]):
        df_interact[col1 + '_' + col2] = df_interact[col1] + '_' + df_interact[col2]

In [64]:
df_interact_cat_dummies = pd.get_dummies(df_interact, 
                                         columns=df_interact.columns[df_interact.dtypes == 'object']).\
                                                                                        drop('y', axis=1)

In [65]:
df_interact_cat_dummies_part, df_interact_cat_dummies_valid = train_test_split(df_interact_cat_dummies, test_size=.3, 
                                                    stratify=y, random_state=17)

In [66]:
np.mean(cross_val_score(forest, df_interact_cat_dummies_part, y_train_part, cv=skf, scoring='roc_auc'))

0.833999698931206

In [67]:
forest.fit(df_interact_cat_dummies_part, y_train_part)
roc_auc_score(y_valid, forest.predict_proba(df_interact_cat_dummies_valid)[:, 1])

0.8618112043382651

In [68]:
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression(random_state=17)

In [69]:
np.mean(cross_val_score(logit, df_interact_cat_dummies_part, y_train_part, cv=skf, scoring='roc_auc'))

0.8610739403953804

In [70]:
logit.fit(df_interact_cat_dummies_part, y_train_part)
roc_auc_score(y_valid, logit.predict_proba(df_interact_cat_dummies_valid)[:, 1])

0.8530284591899913


## Mean Target

Теперь будем использовать технику кодирования категориальных признаков средним значением целевого признака. Это очень мощная техника, правда, надо умело ее использовать – легко переобучиться. Основная идея – для каждого значения категориального признака посчитать среднее значение целевого признака и заменить категориальный признак на посчитанные средние. Правда, считать средние надо на кросс-валидации, а то легко переобучиться.

In [71]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

In [72]:
train_df, y = df.copy(), df['y']
train_df_part, valid_df, y_train_part, y_valid = train_test_split(train_df.drop('y', axis=1), y, 
                                                                  test_size=.3, stratify=y, 
                                                                               random_state=17)

In [73]:
def mean_target_enc(train_df, y_train, valid_df, skf):
        
    glob_mean = y_train.mean()
    train_df = pd.concat([train_df, pd.Series(y_train, name='y')], axis=1)
    new_train_df = train_df.copy()
    
    cat_features = train_df.columns[train_df.dtypes == 'object'].tolist()    

    for col in cat_features:
        new_train_df[col + '_mean_target'] = [glob_mean for _ in range(new_train_df.shape[0])]

    for train_idx, valid_idx in skf.split(train_df, y_train):
        train_df_cv, valid_df_cv = train_df.iloc[train_idx, :], train_df.iloc[valid_idx, :]

        for col in cat_features:
            
            means = valid_df_cv[col].map(train_df_cv.groupby(col)['y'].mean())
            valid_df_cv[col + '_mean_target'] = means.fillna(glob_mean)
            
        new_train_df.iloc[valid_idx] = valid_df_cv
    
    new_train_df.drop(cat_features + ['y'], axis=1, inplace=True)
    
    for col in cat_features:
        means = valid_df[col].map(train_df.groupby(col)['y'].mean())
        valid_df[col + '_mean_target'] = means.fillna(glob_mean)
        
    valid_df.drop(train_df.columns[train_df.dtypes == 'object'], axis=1, inplace=True)
    
    return new_train_df, valid_df

In [74]:
train_mean_target_part, valid_mean_target = mean_target_enc(train_df_part, y_train_part, valid_df, skf)

In [76]:
np.mean(cross_val_score(forest, train_mean_target_part, y_train_part, cv=skf, scoring='roc_auc'))

0.8813941498132323

In [77]:
forest.fit(train_mean_target_part, y_train_part)
roc_auc_score(y_valid, forest.predict_proba(valid_mean_target)[:, 1])

0.9108221780994471

## Mean Target + попарные взаимодействия

In [88]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

In [89]:
train_df, y = df_interact.drop('y', axis=1).copy(), df_interact['y']
train_df_part, valid_df, y_train_part, y_valid = train_test_split(df_interact_cat_dummies, y, 
                                                                  test_size=.3, stratify=y, 
                                                                               random_state=17)

In [90]:
train_mean_target_part, valid_mean_target = mean_target_enc(train_df_part, y_train_part, valid_df, skf)

In [92]:
np.mean(cross_val_score(forest, train_mean_target_part, y_train_part, cv=skf, scoring='roc_auc'))

0.8341390438965304

In [93]:
forest.fit(train_mean_target_part, y_train_part)
roc_auc_score(y_valid, forest.predict_proba(valid_mean_target)[:, 1])

0.8618112043382651

In [94]:
np.mean(cross_val_score(logit, train_mean_target_part, y_train_part, cv=skf, scoring='roc_auc'))

0.8630756627889472

In [95]:
logit.fit(train_mean_target_part, y_train_part)
roc_auc_score(y_valid, logit.predict_proba(valid_mean_target)[:, 1])

0.8530284591899913

## Catboost

В библиотеке Catboost, помимо всего прочего, реализована как раз техника кодирования категориальных значений средним значением целевого признака. Результаты получаются хорошими именно когда в данных много важных категориальных признаков.

In [101]:
from catboost import CatBoostClassifier

In [102]:
ctb = CatBoostClassifier(random_seed=17)

In [103]:
train_df, y = df.drop('y', axis=1), df['y']
train_df_part, valid_df, y_train_part, y_valid = train_test_split(train_df, y, 
                                                                  test_size=.3, stratify=y, 
                                                                  random_state=17)

In [104]:
cat_features_idx = np.where(train_df_part.dtypes == 'object')[0].tolist()

In [109]:
%%time
cv_scores = []
for train_idx, test_idx in skf.split(train_df_part, y_train_part):
    cv_train_df, cv_valid_df = train_df_part.iloc[train_idx, :], train_df_part.iloc[test_idx, :]
    y_cv_train, y_cv_valid = y_train_part.iloc[train_idx], y_train_part.iloc[test_idx]
    
    ctb.fit(cv_train_df, y_cv_train,
        cat_features=cat_features_idx, silent=True);
    
    cv_scores.append(roc_auc_score(y_cv_valid, ctb.predict_proba(cv_valid_df)[:, 1]))

CPU times: user 1min 32s, sys: 7.98 s, total: 1min 40s
Wall time: 32 s


In [110]:
np.mean(cv_scores)

0.9028648621209946

In [111]:
%%time
ctb.fit(train_df_part, y_train_part,
        cat_features=cat_features_idx,silent=True);

CPU times: user 20.8 s, sys: 1.53 s, total: 22.3 s
Wall time: 7.04 s


In [112]:
roc_auc_score(y_valid, ctb.predict_proba(valid_df)[:, 1])

0.9190524989858878

## XGBoost

In [186]:
import xgboost as xgb

In [187]:
data_dmatrix = xgb.DMatrix(data=df_interact_cat_dummies_part, label=y_train_part)

In [188]:
params = {"objective":"binary:logistic"}

In [189]:
cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=5,
                    num_boost_round=50, early_stopping_rounds=20, metrics="auc", as_pandas=True)

In [190]:
cv_results.head()

,train-auc-mean,train-auc-std,test-auc-mean,test-auc-std
0,0.870920,0.015828,0.831096,0.029865
1,0.905074,0.013131,0.854364,0.019743
2,0.925652,0.006637,0.871439,0.014437
3,0.936959,0.005397,0.873073,0.017619
4,0.948667,0.003667,0.876884,0.016534


In [191]:
xgb_class = xgb.XGBClassifier()
xgb_class.fit(train_mean_target_part, y_train_part)
roc_auc_score(y_valid, xgb_class.predict_proba(valid_mean_target)[:, 1])

0.9096692926834475